In [12]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input

# Load the trained model
model = tf.keras.models.load_model('final_save.h5')

# Load the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Emotion labels (keeping 'disgust' but will ignore it in predictions)
emotion_labels = ['Angry', 'Happy', 'fear', 'Neutral', 'Surprise', 'Sad']

# Updated em
# Set the input image size (same as during training)
img_size = (224, 224)

# Start video capture from the default webcam (use '0' for the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        print("Failed to grab frame.")
        break

    # Convert the frame to grayscale (Haar Cascade works best on grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop over the detected faces
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Crop the detected face region
        face = frame[y:y + h, x:x + w]

        # Resize the face to the input size of the model
        face_resized = cv2.resize(face, img_size)

        # Preprocess the face image
        face_array = np.expand_dims(face_resized, axis=0)  # Add batch dimension
        face_array = preprocess_input(face_array)  # Preprocess image for ResNet50

        # Make emotion prediction
        predictions = model.predict(face_array)

        # Exclude the 'disgust' class (assumed to be at index 0)
        predictions = np.delete(predictions, 0, axis=1)  # Remove predictions for disgust
        
        # Get the index of the highest probability emotion from the modified predictions
        emotion_index = np.argmax(predictions)
        emotion = emotion_labels[emotion_index]  # Get corresponding display label

        # Add the emotion label to the frame
        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Emotion Recognition', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close the windows
cap.release()
cv2.destroyAllWindows()
